In [1]:
import sys
import datacube
from datacube.helpers import write_geotiff
from datacube.utils import cog
from datetime import datetime
from datetime import timedelta
import pandas as pd
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from datacube.storage import masking

sys.path.append("Scripts")
from dea_datahandling import load_ard
from dea_plotting import rgb
from dea_plotting import display_map
from dea_bandindices import calculate_indices
from dea_datahandling import wofs_fuser
from dea_dask import create_local_dask_cluster

/env/lib/python3.6/site-packages/datacube/storage/masking.py:4: DeprecationWarning: datacube.storage.masking has moved to datacube.utils.masking
  category=DeprecationWarning)


In [2]:
dc = datacube.Datacube(app="Sentinel_2_burn_ratio")

In [3]:
# Syd
central_lat = -26.789823 
central_lon =  152.796161


# Set the buffer to load around the central coordinates
buffer = 0.5

# Compute the bounding box for the study area
study_area_lat = (central_lat - buffer, central_lat + buffer)
study_area_lon = (central_lon - buffer, central_lon + buffer)

display_map(x=study_area_lon, y=study_area_lat, crs='EPSG:4326', margin=-0.2)

/env/lib/python3.6/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/env/lib/python3.6/site-packages/pyproj/crs/crs.py:294: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(" ".join((projstring, projkwargs)))
/env/lib/python3.6/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: h

In [11]:
# Create a reusable query
query = {
    "x": study_area_lon,
    "y": study_area_lat,
    "time": ('2015-05-01', '2015-06-01'),
    "output_crs": "EPSG:3577",
    "crs":'EPSG:4326',
    "resolution": (-30, 30),
#     "group_by": "solar_day",
#     "fuse_func": wofs_fuser,
}


col = dc.load( product="ga_ls8c_ard_3", 
                group_by="solar_day", 
                measurements=['nbart_blue', 'nbart_green', 'nbart_red', 'nbart_nir', 'nbart_swir_1', 'nbart_swir_2'], 
                **query)

In [12]:
col

<xarray.Dataset>
Dimensions:       (time: 4, x: 3838, y: 4221)
Coordinates:
  * time          (time) datetime64[ns] 2015-05-03T23:40:53.995907 ... 2015-0...
  * y             (y) float64 -3.006e+06 -3.006e+06 ... -3.132e+06 -3.132e+06
  * x             (x) float64 1.976e+06 1.976e+06 ... 2.091e+06 2.091e+06
    spatial_ref   int32 3577
Data variables:
    nbart_blue    (time, y, x) int16 -999 -999 -999 -999 ... -999 -999 -999 -999
    nbart_green   (time, y, x) int16 -999 -999 -999 -999 ... -999 -999 -999 -999
    nbart_red     (time, y, x) int16 -999 -999 -999 -999 ... -999 -999 -999 -999
    nbart_nir     (time, y, x) int16 -999 -999 -999 -999 ... -999 -999 -999 -999
    nbart_swir_1  (time, y, x) int16 -999 -999 -999 -999 ... -999 -999 -999 -999
    nbart_swir_2  (time, y, x) int16 -999 -999 -999 -999 ... -999 -999 -999 -999
Attributes:
    crs:           EPSG:3577
    grid_mapping:  spatial_ref